# <div align="center">Python™ for Earth Scientists</div> 
### <div align="center">Installation Instructions and Verification</div>

This Jupyter notebook contains instructions for installing essential Python™ packages for Earth scientists and tests the functionality of the installed packages. This Jupyter notebook and the orresponding Python™ script have been tested under:  
* Windows 11 Pro (23H2)        Python™ 3.11.7
* Windows 10 Enterprise (21H2) Python™ 3.9.13
* Linux Ubuntu 22.04.2 on WSL2 Python™ 3.11.7 (selected elements)

### First, determine if code is executed from a classic Jupyter Notebook, JupyterLab, or Python™

In [14]:
import psutil
width_output = 78 # number of characters for output 
parent_process = psutil.Process().parent().cmdline()[-1]

str_title = ' Determine Computing Environment '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

if 'jupyter-lab' in parent_process:
    print('Environment: JupyterLab')
    ENVIRONMENT = "Jupyterlab"
elif 'jupyter-notebook' in parent_process:
    print('Environment: Jupyter Notebook')
    ENVIRONMENT = "Jupyternotebook"
elif "spyder-script.py" in parent_process:
    ENVIRONMENT = "Python"
elif parent_process == "-bash":
    print('Environment: Python™ on Linux (command line)')
    ENVIRONMENT = "Python"
elif "anaconda3" in parent_process:
    print('Environment: Python™ on Windows (command line)')
    ENVIRONMENT = "Python"
else:
    print('Environment: Python™')
    ENVIRONMENT = "Python"


---------------------- Determine Computing Environment -----------------------

Environment: JupyterLab


### 1) Get name of computer and operating system information

In [15]:
# get computer name and OS
import os
import socket
import platform

str_title = ' Platform and Operating System '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

print(f'Computer:         {socket.gethostname():s}')
print(f'Operating system: {platform.system():s} {os.name.upper():s} Release: {platform.release():s}')


----------------------- Platform and Operating System ------------------------

Computer:         gs615-aerogeo
Operating system: Windows NT Release: 10


### 2) Get Python™ version

In [16]:
import sys
import jupyterlab

python_version        = sys.version
python_version_detail = python_version.split(" | ",-1) # -1 gets all occurences. can set max number of splits

str_title = ' Python™ Version and Installation '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

print(f'Python™ version:      {sys.version_info.major:d}.{sys.version_info.minor:d}.{sys.version_info.micro:d}')
if len(python_version_detail) == 3:
    print(f'Python™ installation: {python_version_detail[0]:s} {python_version_detail[1]:s}')
    print(f'                      {python_version_detail[2]:s}')
print(f'JupyterLab version:   {jupyterlab.__version__:s}')



---------------------- Python™ Version and Installation ----------------------

Python™ version:      3.9.13
JupyterLab version:   3.4.4


### 3) Verify GeoPandas & GDAL/Python™ installations

In [17]:
"""
    Installation: conda install geopandas
    Installation: conda install geodatasets -c conda-forge # used for testing
"""
import warnings
warnings.filterwarnings("ignore", module="gdal")        # suppresses all warnings from gdal module
warnings.filterwarnings("ignore", module="geodatasets") # suppresses all warnings from geodatasets module
warnings.filterwarnings("ignore", category=DeprecationWarning)

if (ENVIRONMENT == "Jupyterlab" or ENVIRONMENT == "Jupyternotbook") and (sys.version_info.minor > 9):
    print("WARNING: GeoPandas & GDAL/Python™ installations have been verified to work with Xarray in Python™.")
    print("         For reasons that are unclear they produce an errror when executed from JupyterLab or \n         Jupyter Notebook after Xarray is installed.")
    print(f'         => Skipping GeoPandas & GDAL/Python™ verification since code is executed from {ENVIRONMENT:s}.')
elif (ENVIRONMENT == "Jupyterlab" and (sys.version_info.minor <= 9)) or (ENVIRONMENT == "Python"):
    import geodatasets
    import geopandas
    import geopandas as gdp
    from   osgeo import gdal
    from   geodatasets import get_path
    str_title = ' GeoPandas & GDAL/Python™ '
    str_title = str_title.center(width_output, "-")
    print("\n" + str_title + "\n")
    print(f'GeoPandas    version:      {gdp.__version__:s}')
    print(f'GDAL/Python™ version:      {gdal.__version__:s}')
    print(f'geodatasets  version:      {geodatasets.__version__:s}')
    
    # verify GeoPandas
    path_to_data = get_path("nybb")
    gdf = geopandas.read_file(path_to_data)
    
    # verify modules
    f_name_geotiff = r"." + os.sep + "test_data" + os.sep + "GEOTIFF" + os.sep + "IOCAM1B_2019_GR_NASA_20190506-131614.4217.tif"
    
    cambot = gdal.Open(f_name_geotiff)
    cambot_proj = cambot.GetProjection()
    
    if "Polar_Stereographic" in cambot_proj:
        print('GDAL/Python™ verification: GeoTiff projection information contains "Polar_Stereographic"')
    else:
        print('GDAL/Python™ verification: ERROR: GeoTiff projection information could not be read')
    
    # verify GeoPandas
    if hasattr(gdf, 'area'):
      print('GeoPandas    verification: GeoPandas DataFrame has attribute "area"')
      # print(gdf)
    else:
      print('GeoPandas    verification: ERROR: GeoPandas DataFrame has no attribute "area"')
else:
    os.sys.exit("Computing environment is not supported. Abort.")


-------------------------- GeoPandas & GDAL/Python™ --------------------------

GeoPandas    version:      0.12.2
GDAL/Python™ version:      3.0.2
geodatasets  version:      2023.12.0
GDAL/Python™ verification: GeoTiff projection information contains "Polar_Stereographic"
GeoPandas    verification: GeoPandas DataFrame has attribute "area"


### 4) Verify OpenCV installation

In [18]:
"""
    see: https://opencv.org/get-started/
    pip3 install opencv-python
    -> Successfully installed opencv-python-4.9.0.80
    TODO: unclear if .dll needs to be copied as described in link above 
"""
import cv2 as cv2
import pathlib

if platform.system() == "Windows":
    f_name_jpg = pathlib.Path(r"./test_data/JPEG/IOCAM0_2019_GR_NASA_20190506-131614.4217.jpg")
elif platform.system() == 'Linux':
    f_name_jpg = pathlib.Path(r"./test_data/JPEG/IOCAM0_2019_GR_NASA_20190506-131614.4217.jpg")

image_bgr  = cv2.imread(str(f_name_jpg))
# image      = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
img_size   = image_bgr.shape
 
str_title = ' OpenCV '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'OpenCV version:      {cv2.__version__:s}')
print(f'Test image:          {img_size[0]:d} × {img_size[1]:d} pixels')


----------------------------------- OpenCV -----------------------------------

OpenCV version:      4.8.0
Test image:          3264 × 4896 pixels


### 5) Verify Pytorch, Torchvision, and Torchaudio CPU only installation

In [19]:
"""
The SAM Python™ module requires PyTorch and TorchVision. The SAM installation instructions recommend installing
both packages with CUDA support, however, if that causes error messages the solution is often to install
both packages without CUDA support.

    Recommended installation with CUDA support:
    > conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
    Installation without CUDA support (CPU only):
    > conda install pytorch torchvision torchaudio cpuonly -c pytorch
    To install the SAM Python™ module use (see below):
    pip install git+https://github.com/facebookresearch/segment-anything.git

"""
# verify the installation and check for CUDA support:
import torch
import torchvision

# determine which processing unit to use
if torch.cuda.is_available():
    processing_unit = "cuda" # use graphics processing unit (GPU)
else:    
    processing_unit = "cpu"  # use central processing unit (CPU)
    
str_title = ' PyTorch & TorchVision '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'PyTorch     version: {torch.__version__:s}')
print(f'Torchvision version: {torchvision.__version__:s}')
print(f'Processing  support: {processing_unit.upper():s}')


--------------------------- PyTorch & TorchVision ----------------------------

PyTorch     version: 2.3.0
Torchvision version: 0.18.0
Processing  support: CPU


### 6) Verify Segment Anything Model (SAM) installation and import

In [20]:
"""
    The SAM Python™ module requires PyTorch and TorchVision (see Section 5 above).
    To install the SAM Python™ module use:
    pip install git+https://github.com/facebookresearch/segment-anything.git
"""
import segment_anything
    
str_title = ' Segment Anything Model (SAM) '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

modulename = 'segment_anything'
if modulename not in sys.modules:
    print(f'Segment Anything: ERROR: {modulename:s} module is unavailable')
else:
    print(f'Segment Anything: {modulename:s} module is imported and in the sys.modules dictionary')


------------------------ Segment Anything Model (SAM) ------------------------

Segment Anything: segment_anything module is imported and in the sys.modules dictionary


### 7) Verify PYPRØJ installation

PYPRØJ is a Python™ interface to PROJ, a cartographic projections and coordinate transformations library

In [21]:
"""
    PYPRØJ and shapely are installed as part of geopandas. If geopandas is not installed use:
    pip install pyproj
    Shapely is a Python™ package that uses the GEOS library to perform set-theoretic operations
    on planar features. 
"""
import pyproj
import shapely
import numpy as np
from  shapely import Point

# verify PYPRØJ
# EPSG:3413 NSIDC Sea Ice Polar Stereographic North/WGS-84 used for Greenland
# EPSG:4326 WGS84 - World Geodetic System 1984, used in GPS 
geo2xy = pyproj.Transformer.from_crs(4326,3413)
xy2geo = pyproj.Transformer.from_crs(3413,4326)
xy     = geo2xy.transform(70.0, -45.0)
lonlat = xy2geo.transform(xy[0],xy[1])

# verify Shapely
patch = Point(0.0, 0.0).buffer(1.0)
path_area = patch.area # result should be Pi

str_title = ' PYPRØJ & Shapely '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

print(f'PYPRØJ       version: {pyproj.proj_version_str:s}')
print(f'Shapely      version: {shapely.__version__:s}')
if lonlat[1] == -45:
    print('PYPRØJ  verification: projected geographic coordinates to polar stereographic')
if (np.isfinite(path_area)) & (path_area > 0):
    print('Shapely verification: calculated buffer area around point geometry')


------------------------------ PYPRØJ & Shapely ------------------------------

PYPRØJ       version: 6.2.1
Shapely      version: 2.0.1
PYPRØJ  verification: projected geographic coordinates to polar stereographic
Shapely verification: calculated buffer area around point geometry


### 8) Verify PyCRS installation

PyCRS is a pure Python GIS package for reading, writing, and converting between various common coordinate reference system (CRS) string and data source formats.

In [22]:
import pycrs 
str_title = ' PyCRS GIS '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")

print(f'PyCRS version: {pycrs.__version__:s}')



--------------------------------- PyCRS GIS ----------------------------------

PyCRS version: 1.0.2


### 9) Verify PyMap3D  installation

PyMap3D is a Python™ (optional Numpy) toolbox for 3D geographic coordinate transformations

In [23]:
"""
    PyMap3D is a Python™ (optional Numpy) toolbox for 3D geographic coordinate transformations and geodesy.
    It supports various coordinate systems, ellipsoids, and Vincenty functions, and has a similar syntax 
    to the MATLAB®  Mapping Toolbox.
    Installation: conda install -c conda-forge pymap3d
"""
import pymap3d

str_title = ' PyMap3D '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'PyMap3D version: {pymap3d.__version__:s}') 


---------------------------------- PyMap3D -----------------------------------

PyMap3D version: 3.0.1


### 10) Verify Python™ implementation of Scientific Colour Maps

In [24]:
"""
    Python™ implementation of the Scientific Colour Maps version 8.0 (2023-06-14).
    Installation: conda install -c conda-forge cmcrameri
"""
import cmcrameri

str_title = ' Python™ implementation of the Scientific Colour Maps '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'cmcrameri version: {cmcrameri.__version__:s}') 



------------ Python™ implementation of the Scientific Colour Maps ------------

cmcrameri version: 1.4


### 11) Verify laspy & lazrs-python installation

In [25]:
"""
    laspy is a Python™ module that reads and writes LAS and LAZ files, which are common formats for lidar pointcloud and full waveform data.
    Installation: conda install -c conda-forge laspy
    See also: https://github.com/LAStools/LAStools
"""
import laspy

str_title = ' laspy & lazrs-python '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'laspy version: {laspy.__version__:s}') 


---------------------------- laspy & lazrs-python ----------------------------

laspy version: 2.4.1


### 12) Verify Xarray installation

In [26]:
# Xarray
"""
    Xarray is a Python™ library that provides a common interface for working with n-dimensional arrays and datasets.
    Installation: conda install -c conda-forge xarray dask netCDF4 bottleneck
"""
import xarray

str_title = ' Xarray '
str_title = str_title.center(width_output, "-")
print("\n" + str_title + "\n")
print(f'Xarray version: {xarray.__version__:s}')


----------------------------------- Xarray -----------------------------------

Xarray version: 0.20.1
